In [1]:
import numpy as np
from scipy.stats import norm

In [3]:
def _gbs(S, K, T, rd, b, sigma):
    T_sqrt = np.sqrt(T)
    d1 = (np.log(S / K) + (b + (sigma * sigma) / 2) * T) / (sigma * T_sqrt)
    d2 = d1 - sigma * T_sqrt

    value = S * np.exp((b - rd) * T) * norm.cdf(d1) - K * np.exp(-rd * T) * norm.cdf(d2)
    delta = np.exp((b - rd) * T) * norm.cdf(d1)
    gamma = np.exp((b - rd) * T) * norm.pdf(d1) / (S * sigma * T_sqrt)
    theta = -(S * sigma * np.exp((b - rd) * T) * norm.pdf(d1)) / (2 * T_sqrt) \
            - (b - rd) * S * np.exp((b - rd) * T) * norm.cdf(d1) \
            - rd * K * np.exp(-rd * T) * norm.cdf(d2)
    vega = np.exp((b - rd) * T) * S * T_sqrt * norm.pdf(d1)
    rho = K * T * np.exp(-rd * T) * norm.cdf(d2)

    return value, delta, gamma, theta, vega, rho


def garman_kohlhagen(S, K, T, rd, rf, sigma):
    """
    Args:
        S (float): Price of underlying asset. (domestic/foreign)
        K (float): Strike price.
        T (float): Time to expiration in years. 1 for one year, 0.5 for 6 months.
        rd (float): Risk free rate.
        rf (float): Risk free rate of the foreign currency.
        sigma (float): Implied volatility of underlying asset.

    Returns:
        value (float): Price of the option.
        delta (float): First derivative of value with respect to price of underlying.
        gamma (float): Second derivative of value w.r.t price of underlying.
        theta (float): Minus first derivative of value w.r.t. time to expiration.
        vega (float): First derivative of value w.r.t. implied volatility.
        rho (float): First derivative of value w.r.t. risk free rates.
    """
    b = rd - rf
    return _gbs(S, K, T, rd, b, sigma)

In [4]:
spot = 1.0934
strike = 0.98406 # the first type of call
time = 1
rate_domestic = 5.35/100
rate_foreign = 3.662/100
volatility = 5.56/100

d1 = dict(zip(('Value','Delta','Gamma','Theta', 'Vega','Rho'),
             garman_kohlhagen(spot, strike, time, rate_domestic, rate_foreign, volatility)))
for key, value in d1.items():
    print(f'{key} = {value}')

Value = 0.12155808726348472
Delta = 0.9515149698258667
Gamma = 0.5306560547748682
Theta = -0.01203896356672491
Vega = 0.035273296955144307
Rho = 0.918828380744118


In [5]:
spot = 1.0934
strike = 1.20274 # the second type of call
time = 1
rate_domestic = 5.35/100
rate_foreign = 3.662/100
volatility = 5.56/100

d2 = dict(zip(('Value','Delta','Gamma','Theta', 'Vega','Rho'),
             garman_kohlhagen(spot, strike, time, rate_domestic, rate_foreign, volatility)))
for key, value in d2.items():
    print(f'{key} = {value}')

Value = 0.0021826255289070506
Delta = 0.08036338284204937
Gamma = 2.4318034145094867
Theta = -0.0058601855876780655
Vega = 0.16164467210860026
Rho = 0.08568669727058974


In [6]:
C1, C2 = d1['Value'], d2['Value'] # in USD
delta_1, delta_2 = d1['Delta'], d2['Delta']
gamma_1, gamma_2 = d1['Gamma'], d2['Gamma']
theta_1, theta_2 = d1['Theta'], d2['Theta']
a = np.array([[delta_1, -delta_2, 1],[gamma_1, -gamma_2, 0],[1, 1, 1]])
b = np.array([0, 0, 1])
x = np.linalg.solve(a, b)
n1, n2, n3 = x # optimal proportions of capital allocation
print(n1, n2, n3)
print(n1/n2)
print(abs(n3)/abs(n2))

3.518196275889422 0.7677233054891869 -3.2859195813786095
4.582635763086099
4.280083146993759


In [7]:
V = 100000 # in USD
N2 = V / (spot*abs(n3)/n2 - C2 + C1*n1/n2)
print(N2)

19103.2304604295


In [12]:
N1 = n1/n2*N2
N3 = abs(n3)/n2*N2
print(N1)
print(N2)
print(N3)

87543.14709843995
19103.2304604295
81763.41474682212


In [13]:
print(np.isclose(V - N1*C1 + N2*C2 - abs(N3)*spot, 0)) # checking that everytin came together

True


In [14]:
print(abs(N3)*spot) # the number of dollars kept in our portfolio

89400.1176841753


In [15]:
Theta = N1*theta_1-N2*theta_2 # portfolio theta
print(Theta)
print(Theta/252) # theta in trading days

-941.9802826122564
-3.738016994493081
